In [1]:
from IPython.display import clear_output
from ipywidgets.widgets import IntSlider,FloatSlider,interact
from itertools import product
from random import randint
import time
from pprint import pprint

def vector_value(vector:str):
    return vector.count('1')

def vector_and(a,b):
    new_vector = []
    for index,i in enumerate(a):
        new_vector.append(
            i if i == b[index] else '0'
        )
    return ''.join(new_vector)   

def generate_vectors(n):
    return list(
        map(
            lambda a: ''.join(list(a)),
            product(['0','1'],repeat=n)
        )
    )

def random_vectors(vectors,count):
    result = []
    for i in range(count):
        result.append(vectors[randint(0,len(vectors)-1)])
    return result

def deduplicate_clusters(clusters):
    result = {}
    for cluster in clusters:
        result[cluster[0]] = set()

    for cluster in clusters:
        result[cluster[0]] |= set(cluster[1]) 
    return result

def compare_clusters(current,prev):
    current_set = set(current)
    prev_set = set(prev)
    return not current_set == prev_set

def art1_step(prototype_list,attribute_list,beta,rho,verbose):
    
    def beta_check(proto,attr):
        t1 = vector_value(vector_and(proto,attr))
        t2 = vector_value(proto) + beta
        t3 = vector_value(attr) / (beta+len(proto))
        return (t1/t2) > t3

    def rho_check(proto,attr):
        t1 = vector_value(vector_and(proto,attr))
        t2 = vector_value(attr)
        return (t1/t2) > rho

    clusters = []
    for i in prototype_list:
        clusters.append([i,[]])
    
    for attribute in attribute_list:
        flag = True
        for cluster in clusters:
            if beta_check(cluster[0],attribute):
                if rho_check(cluster[0],attribute):
                    if verbose:
                        print('Добавление признака {} в прототип {}'.format(attribute,cluster[0]))
                    cluster[0] = vector_and(cluster[0],attribute)
                    cluster[1].append(attribute)
                    flag = False
                    break
        if flag: 
            clusters.append([attribute,[attribute]])
            if verbose:
                print('Добавление прототипа {}'.format(attribute))
    return clusters

d_widget = IntSlider(min=3,max=15,step=1,value=4)
n_widget = IntSlider(min=2,max=16,step=1,value=10)

def d_widget_update(*args):
    n_widget.max = d_widget.value**2-1

d_widget.observe(d_widget_update,'value')

@interact(
    d=d_widget,
    rho=FloatSlider(min=0.01,max=1,step=0.01),
    beta=IntSlider(min=1,max=10,step=1),
    n=n_widget,
    verbose = True
)
def art1(d,rho,beta,n,verbose):
    vectors = generate_vectors(d)
    e_vectors = random_vectors(vectors,n)
    p_vectors = []
    prev_p_vectors = []
    clusters = []

    clear_output()

    p_vectors.append(e_vectors[0])
    if verbose:
        print('Векторы-прототипы {}'.format(p_vectors))
    
    print('Старт программы')
    while compare_clusters(p_vectors,prev_p_vectors):
        prev_p_vectors = p_vectors
        clusters = art1_step(p_vectors,e_vectors,beta,rho,verbose)
        
        p_vectors = []
        for cluster in clusters:
            p_vectors.append(cluster[0])

        clusters = deduplicate_clusters(clusters)

        if verbose:
            print('Кластеры:')
            for cluster in clusters:
                print('Кластер:')
                pprint(cluster)

            print('Векторы прототипы:')
            print('Предыдущий:')
            pprint(prev_p_vectors)
            print('Текущий')
            pprint(p_vectors)
    print('Векторы прототипов не изменились. Выход из цикла.')
    pprint(clusters) 


interactive(children=(IntSlider(value=4, description='d', max=15, min=3), FloatSlider(value=0.01, description=…